In [1]:
import tensorflow

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import keras

Using TensorFlow backend.


In [3]:

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [59]:
img_width, img_height = 256, 256
train_data_dir = "/data/train"
validation_data_dir = "data/train"
nb_train_samples = 602813/10
nb_validation_samples = 66491/20 
batch_size = 16
epochs = 50

In [49]:
nr_categories = 997
batch_size = 16
epochs = 50

In [50]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [51]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [52]:
# Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
for layer in model.layers[:]:
    print(layer.name)
    layer.trainable = False

input_2
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool


In [53]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

In [54]:
model_final = Model(input = model.input, output = predictions)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [55]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [56]:
# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  from ipykernel import kernelapp as app


In [57]:
train__val_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.1)

In [14]:
#validation_datagen = ImageDataGenerator(rescale = 1./255)

In [22]:
train_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(256, 256),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

Found 602813 images belonging to 997 classes.


In [23]:
validation_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(256, 256),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')

Found 66491 images belonging to 997 classes.


In [25]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [ ]:
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

In [60]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, validation_data=<keras.pre..., callbacks=[<keras.ca..., steps_per_epoch=3767.0, validation_steps=3324.55)`


Epoch 1/50
3767/3767 [==============================] - 1792s 476ms/step - loss: 6.3950 - acc: 0.0082 - val_loss: 6.1415 - val_acc: 0.0200

Epoch 00001: val_acc improved from -inf to 0.02004, saving model to vgg16_1.h5
Epoch 2/50
3767/3767 [==============================] - 1715s 455ms/step - loss: 6.1094 - acc: 0.0188 - val_loss: 5.9100 - val_acc: 0.0412

Epoch 00002: val_acc improved from 0.02004 to 0.04122, saving model to vgg16_1.h5
Epoch 3/50
3767/3767 [==============================] - 1706s 453ms/step - loss: 5.8847 - acc: 0.0336 - val_loss: 5.6812 - val_acc: 0.0558

Epoch 00003: val_acc improved from 0.04122 to 0.05575, saving model to vgg16_1.h5
Epoch 4/50
3767/3767 [==============================] - 1699s 451ms/step - loss: 5.6595 - acc: 0.0521 - val_loss: 5.4615 - val_acc: 0.0823

Epoch 00004: val_acc improved from 0.05575 to 0.08231, saving model to vgg16_1.h5
Epoch 5/50
3767/3767 [==============================] - 1699s 451ms/step - loss: 5.4691 - acc: 0.0671 - val_loss: 5

KeyboardInterrupt: 

In [ ]:
!ls